In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import json
import optuna
import ast
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

In [44]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [45]:
data = pd.read_pickle(path + r'/data/Volve/Volve_cleaned_prepared.pkl')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101892,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.114759,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082297,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [46]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5769: 'NO 15/9-F-5 AH',
 7078: 'NO 15/9-F-11 H',
 5351: 'NO 15/9-F-14 H',
 5599: 'NO 15/9-F-12 H',
 7289: 'NO 15/9-F-15 D',
 7405: 'NO 15/9-F-1 C'}

In [47]:
data.columns

Index(['well_name', 'WELL_BORE_CODE', 'date', 'prod_hrs', 'bhp', 'bht',
       'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'oil_rate',
       'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long',
       'Rolling_short', 'Rolling_medium', 'Rolling_long', 'Lag_short',
       'Lag_medium', 'Lag_long'],
      dtype='object')

In [48]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [49]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [50]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                     -0.0
bht                      0.0
dp_tubing               -0.0
AVG_ANNULUS_PRESS       -0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                 -0.0
water_vol                0.0
linear_regressor        -0.0
EMA_short                0.0
EMA_medium               0.0
EMA_long                -0.0
Rolling_short           -0.0
Rolling_medium           0.0
Rolling_long             0.0
oil_rate                 0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [51]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,-0.102310,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,-0.102310,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,0.177706,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,-0.042238,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,0.028674,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,...,-0.692802,-0.702316,-0.715245,-0.692202,-0.703021,-0.724022,-0.683109,"[-0.6852415797773639, -0.6943429268335604, -0....","[-0.6933422169244923, -0.687878486910018, -0.6...","[-0.7133929373653101, -0.7128670168291575, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,...,-0.689514,-0.699464,-0.713067,-0.690182,-0.699482,-0.722275,-0.677835,"[-0.6943429268335604, -0.6840874763785847, -0....","[-0.687878486910018, -0.6852415797773639, -0.6...","[-0.7128670168291575, -0.704496114962062, -0.6..."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,...,-0.687737,-0.697357,-0.711216,-0.68894,-0.696914,-0.720063,-0.679069,"[-0.6840874763785847, -0.6831086798251896, -0....","[-0.6852415797773639, -0.6943429268335604, -0....","[-0.704496114962062, -0.6974619277910209, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,...,-0.68641,-0.695555,-0.7095,-0.685783,-0.695013,-0.71813,-0.678646,"[-0.6831086798251896, -0.6778348655598816, -0....","[-0.6943429268335604, -0.6840874763785847, -0....","[-0.6974619277910209, -0.6908660077334403, -0...."


In [52]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [53]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [54]:
# Get the numerical index of the column 'oil_rate'
column_name = 'oil_rate'
column_index = data[feature_columns].columns.get_loc(column_name)
column_index

19

In [55]:
batch_size = 96
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window

In [56]:
# sequences = []
# for well_name in data['well_name'].unique():
#     well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
#     for i in range(len(well_data) - training_gap + 1):
#         single_features = np.array(well_data.iloc[i:i + days_window][single_feature_columns])
#         multiple_features = np.array(well_data.iloc[i:i + days_window][multiple_feature_columns])
#         multiple_flattened = []
#         for k in multiple_features:
#            flattened_list = [item for sublist in k for item in sublist]
#            multiple_flattened.append(flattened_list)
#         multiple_flattened = np.array(multiple_flattened)
#         inputs = np.concatenate((single_features, multiple_flattened), axis=1)
#         targets = well_data.iloc[i + days_window:i + days_window + predictions_days][target_column].values
#         inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
#         targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
#         inputs_index = list(well_data.iloc[i:i + days_window]['index'])
#         targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
#         sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
#         if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
#           print('violation')

# dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
# dataset.to_pickle(path + "/data/Volve/all_data.pkl")
# dataset

In [57]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [58]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [59]:
# train_wells = []
# test_wells = []
# for i in well_info.keys():
#     well = dataset[dataset['well_name'] == i]
#     train_set_length = int((train_size + val_size) * len(well))
#     train_wells.append(well[:train_set_length + 1])
#     test_set_length = int(test_size * len(well))
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# temp_data = pd.concat(train_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)


# data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

# data_train.reset_index(drop=True, inplace=True)
# data_val.reset_index(drop=True, inplace=True)
# data_test.reset_index(drop=True, inplace=True)

# data_train.to_pickle(path + "/data/Volve/data_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_test.pkl")


In [60]:
all_data = pd.read_pickle(path + "/data/Volve/all_data.pkl")
data_train = pd.read_pickle(path + "/data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "/data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "/data/Volve/data_test.pkl")

In [61]:
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [62]:
check = True
for i, j in well_info.items():
    if len(all_data[all_data['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [63]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [64]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [65]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu' 
device

'mps'

In [66]:
eval_interval = 500
test_eval_interval = 500
test_iterations = 7500
warmup_steps = 6000 # hyper-parameters for hyperprameter finetuning

In [67]:
class LSTM(nn.Module):
    def __init__(self, input_length, hidden_length, output_length, num_layers, dropout=0):
        super().__init__()
        self.input_length = input_length
        self.hidden_length = hidden_length
        self.output_length = output_length
        self.init_hidden = nn.Parameter(torch.zeros(num_layers, 1, hidden_length))
        self.init_cell = nn.Parameter(torch.zeros(num_layers, 1, hidden_length))
        self.num_layers = num_layers
        self.lstm_cell = nn.LSTM(self.input_length, self.hidden_length, self.num_layers, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_length, self.output_length)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, targets=None):
        B, T, C = x.shape
        init_hidden = self.init_hidden.repeat(1, B, 1)
        init_cell = self.init_cell.repeat(1, B, 1)
        cell_output, _ = self.lstm_cell(x, (init_hidden, init_cell))
        cell_dropout = self.dropout(cell_output)
        logits = self.output_layer(cell_dropout[:, -1, :])
        if targets is None:
            loss = None
        else:
            B, T = logits.shape
            logits = logits.view(B*T)
            targets = targets.view(B*T)
            loss = F.mse_loss(logits, targets)
        return logits, loss

    def get_hyperparameters(self):
        return {'input_length': self.input_length, 'hidden_length': self.hidden_length, 'output_length': self.output_length, 'num_layers': self.num_layers, 'dropout': self.dropout}

In [68]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

train_dataset = TimeSeriesDataset(data_train)
val_dataset = TimeSeriesDataset(data_val)
test_dataset = TimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [69]:
class VisualizationTimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        well_name = self.sequences_df.iloc[idx]['well_name']
        inputs_dates = self.sequences_df.iloc[idx]['inputs_dates']
        targets_dates = self.sequences_df.iloc[idx]['targets_dates']

        return well_name, inputs_dates, targets_dates, torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

visualization_test_dataset = VisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [70]:
input_length = data_train['inputs'][0].shape[1]
output_length = predictions_days

In [76]:
@torch.no_grad()
def estimate_loss(model): # Function to estimate train and val loss while training
    model.eval()  # Set model to evaluation mode
    out = {}
    loaders = {'train': train_loader, 'val': val_loader}
    for mode in ['train', 'val']:
        losses = []
        # Iterate over the DataLoader for a fixed number of batches
        for X_batch, Y_batch in loaders[mode]:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model to get loss
            losses.append(loss.item())  # Add value to losses list
        out[mode] = torch.tensor(losses).mean().item()  # Get mean of losses tensor and save it
    model.train()  # Put model back in train mode
    return out

@torch.no_grad()
def save_params(model, optimizer, scheduler, name=""): # function to save parameters, optimizer and scheduler states
    config_path = path + r'/Models/Global/LSTM/Model/State/'
    model_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if name != "":
        name = f'{name}_'
    torch.save(model.state_dict(), config_path + f'{name}state_params.pth') # to save parameters
    torch.save(optimizer.state_dict(), config_path + f'{name}state_optimizer.pth') # to save optimizer state
    torch.save(scheduler.state_dict(), config_path + f'{name}state_scheduler.pth') # to save scheduler states
    torch.save(model, model_path + f'{name}complete_params.pth')
    torch.save(optimizer, model_path + f'{name}complete_optimizer.pth') 
    torch.save(scheduler, model_path + f'{name}complete_scheduler.pth')

def get_next_log_file(log_dir, prefix="training_run_"):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        return os.path.join(log_dir, f"{prefix}1.txt")
    existing_files = os.listdir(log_dir)
    log_numbers = []
    for filename in existing_files:
        match = re.match(rf"{prefix}(\d+).txt", filename)
        if match:
            log_numbers.append(int(match.group(1)))
    if log_numbers:
        next_number = max(log_numbers) + 1
    else:
        next_number = 1
    return os.path.join(log_dir, f"{prefix}{next_number}.txt")

def log_to_file(log_file, message):
    with open(log_file, 'a') as f:
        f.write(message + '\n')

def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

@torch.no_grad()
def evaluate(mode, model):
    loader_dict = {'train': train_loader, 'val': val_loader, 'test': test_loader}
    loader = loader_dict[mode]
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_targets = []
    test_losses = []
    for X_batch, Y_batch in loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model to get loss
        test_losses.append(loss.item())  # Collect test loss
        B, T = Y_batch.shape
        Y_batch = Y_batch.view(B*T)
        # Convert predictions and targets to NumPy arrays
        all_preds.append(logits.cpu().numpy())  # .cpu() moves tensor to CPU, .numpy() converts to NumPy array
        all_targets.append(Y_batch.cpu().numpy())
    # Concatenate all predictions and targets
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    # Calculate metrics
    metrics = calculate_metrics(all_targets, all_preds)
    metrics['Loss'] = np.mean(test_losses)
    print(f"Metrics on {mode} set:\n")
    print(f"Loss is {metrics['Loss']:.4f}")
    print(f"MAE is {metrics['MAE']:.4f}")
    print(f"RMSE is {metrics['RMSE']:.4f}")
    print(f"R² is {metrics['R2']:.4f}")
    print("\n")
    return metrics

@torch.no_grad()
# Define an objective function for Optuna
def objective(trial):
    vector_length = trial.suggest_int('vector_length', 16, 2048, step=16)
    num_layers = trial.suggest_int('num_layers', 1, 5) # maybe increase it to 6
    dropout = trial.suggest_float('dropout', 0.1, 0.6)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 0.95, 0.99)
    model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler
    train_loader_iter = iter(train_loader)
    # Training loop
    for i in range(test_iterations):
        model.train()
        if (i % test_eval_interval == 0 or i == test_iterations - 1):
            losses = estimate_loss(model)
            trial.report(losses['val'], i)
            if i != 0:
                scheduler.step()
            # if i >= warmup_steps and trial.should_prune():
            #     raise optuna.TrialPruned()
        try:
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)
            
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    val_metrics = evaluate('val', model)
    return val_metrics['Loss']

def load_model(name=""): 
    config_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if name != "":
        name = f'{name}_'
    model = torch.load(config_path + f'{name}complete_params.pth')
    optimizer = torch.load(config_path + f'{name}complete_optimizer.pth')
    scheduler = torch.load(config_path + f'{name}complete_scheduler.pth')
    return model, optimizer, scheduler

def get_oil_history(date, well_name):
    well = data[data["well_name"] == int(well_name)].reset_index(drop=True)
    well = well[pd.to_datetime(well['date']) < date]
    return well['oil_rate']

def rename(config_path, model_path, old_name, new_name):
    os.rename(config_path + f'{old_name}state_params.pth', config_path + f'{new_name}state_params.pth')
    os.rename(config_path + f'{old_name}state_optimizer.pth', config_path + f'{new_name}state_optimizer.pth')
    os.rename(config_path + f'{old_name}state_scheduler.pth', config_path + f'{new_name}state_scheduler.pth')
    os.rename(model_path + f'{old_name}complete_params.pth', model_path + f'{new_name}complete_params.pth')
    os.rename(model_path + f'{old_name}complete_optimizer.pth', model_path + f'{new_name}complete_optimizer.pth')
    os.rename(model_path + f'{old_name}complete_scheduler.pth', model_path + f'{new_name}complete_scheduler.pth')
    
def label_model_files(old_name="", new_name=""):
    if old_name != "":
        old_name = f'{old_name}_'
    if new_name != "":
        new_name = f'{new_name}_'
    config_path = path + r'/Models/Global/LSTM/Model/State/'
    model_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if os.path.exists(config_path + f'{new_name}state_params.pth'):
        try:
            user_choice = int(input("File with new name already exists. Enter 1 if you want to overwrite the files. Otherwise press 2: "))
        except ValueError:
            print("Invalid input. Please enter 1 or 2.")
            return
        if user_choice == 1:
            try:
                rename(config_path, model_path, old_name, new_name)
                print("Files have been renamed successfully.")
            except FileNotFoundError:
                print("One or more files not found.")
        elif user_choice == 2:
            print("Operation cancelled by the user.")
        else:
            print("Invalid choice. Operation cancelled.")
    else:
        try:
            rename(config_path, model_path, old_name, new_name)
            print("Files have been renamed successfully.")
        except FileNotFoundError:
            print("One or more files not found.")

def predictions_with_history_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions_with_history.pdf")
    with torch.no_grad():
        model.eval()
        for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            choice = random.randint(0, len(X_batch) - 1)
            X_batch = X_batch[choice].unsqueeze(0)
            Y_batch = Y_batch[choice]
            well_name = well_name[choice]
            inputs_dates = [i[choice] for i in inputs_dates]
            targets_dates = [i[choice] for i in targets_dates]

            history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
            logits, loss = model(X_batch, Y_batch)
            label = Y_batch.cpu().numpy()
            prediction = logits.cpu().numpy()
            rmse = np.sqrt(mean_squared_error(label, prediction))
            # Plotting
            forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
            plt.figure(figsize=(12, 6))
            plt.plot(range(1, len(history) + 1), history, label='Input Values')
            plt.plot(forecast_index, prediction, label='Forecasted Values')
            plt.plot(forecast_index, label, label='Actual Values')
            # Determine the interval for ticks to have approximately 10 ticks
            tick_interval = max(10, round((len(history) + predictions_days + 1) / 10 / 10) * 10)
            # Generate tick positions
            tick_positions = np.arange(0, len(history) + predictions_days + 1, tick_interval)
            tick_positions[0] = 1  # Start tick at 1 instead of 0

            # Generate tick labels
            tick_labels = tick_positions
            plt.xticks(ticks=tick_positions, labels=tick_labels)
            plt.axvline(x=len(history), color='r', linestyle='--', label='Forecast Start')
            plt.xlabel('Day')
            plt.ylabel('Value')
            plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE = {rmse:.4f}')
            plt.legend()
            plt.grid(True)
            # Save the current figure to the PDF
            pdf.savefig()
            plt.close()
    pdf.close()

def predictions_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions.pdf")
    with torch.no_grad():
        model.eval()
        for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            choice = random.randint(0, len(X_batch) - 1)
            X_batch = X_batch[choice].unsqueeze(0)
            Y_batch = Y_batch[choice]
            well_name = well_name[choice]
            logits, loss = model(X_batch, Y_batch)
            inputs = X_batch[0][:, column_index].cpu().numpy()
            label = Y_batch.cpu().numpy()
            prediction = logits.cpu().numpy()
            rmse = np.sqrt(mean_squared_error(label, prediction))
            # Plotting
            forecast_index = range(len(inputs) + 1, len(inputs) + predictions_days + 1)
            plt.figure(figsize=(12, 6))
            plt.plot(range(1, len(inputs) + 1), inputs, marker='o', label='Input Values')
            plt.plot(forecast_index, prediction, marker='o', label='Forecasted Values')
            plt.plot(forecast_index, label, marker='o', label='Actual Values')
            plt.xticks(ticks=np.arange(1, len(inputs) + predictions_days + 1), labels=np.arange(1, len(inputs) + predictions_days + 1))
            plt.axvline(x=len(inputs), color='r', linestyle='--', label='Forecast Start')
            plt.xlabel('Day')
            plt.ylabel('Value')
            plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE: {rmse:.4f}')
            plt.legend()
            plt.grid(True)
            # Save the current figure to the PDF
            pdf.savefig()
            plt.close()
    pdf.close()

def all_predictions_graphs(model, data, data_test, path):
    well_dataloaders = {}
    pdf = PdfPages(path + "/all_predictions.pdf")
    with torch.no_grad():
        for well_name in well_info.keys():
            well = data[data['well_name'] == well_name].reset_index(drop=True)
            fig, ax = plt.subplots(figsize=(15, 6))
            ax.plot(well.index, well['oil_rate'], label='Well Production')
            ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
            ax.legend()
            well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
            well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
            total_loss = 0
            total_rmse = 0
            for _, inputs_dates, targets_dates, X_batch, Y_batch in well_dataloaders[well_name]:
                inputs_dates = np.array(inputs_dates).T[0]
                targets_dates = np.array(targets_dates).T[0]
                history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
                X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
                logits, loss = model(X_batch, Y_batch)
                label = Y_batch.cpu().numpy().reshape(-1)
                prediction = logits.cpu().numpy()
                rmse = np.sqrt(mean_squared_error(label, prediction))
                total_loss += loss
                total_rmse += rmse
                forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
                ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
                if i == 0:
                    ax.legend()
            avg_loss = total_loss / len(well_dataloaders[well_name]) 
            avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
            ax.set_xlabel('Day')
            ax.set_ylabel('Value')
            ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
            pdf.savefig(fig)
            plt.close()
    pdf.close()

def all_predictions_with_gaps_graphs(model, data, data_test, path):
    well_plots = {}
    well_dataloaders = {}   
    pdf = PdfPages(path + "/all_predictions_with_gaps.pdf")
    with torch.no_grad():
        for well_name in well_info.keys():
            well = data[data['well_name'] == well_name].reset_index(drop=True)
            fig, ax = plt.subplots(figsize=(15, 6))
            ax.plot(well.index, well['oil_rate'], label='Well Production')
            ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
            ax.legend()
            well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
            well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
            total_loss = 0
            total_rmse = 0
            for i, (_, inputs_dates, targets_dates, X_batch, Y_batch) in enumerate(well_dataloaders[well_name]):
                if i % predictions_days == 0 or i == len(well_dataloaders[well_name]) - 1:
                    inputs_dates = np.array(inputs_dates).T[0]
                    targets_dates = np.array(targets_dates).T[0]
                    history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
                    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
                    logits, loss = model(X_batch, Y_batch)
                    label = Y_batch.cpu().numpy().reshape(-1)
                    prediction = logits.cpu().numpy()
                    rmse = np.sqrt(mean_squared_error(label, prediction))
                    total_loss += loss
                    total_rmse += rmse
                    forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
                    ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
                    if i == 0:
                        ax.legend()
            avg_loss = total_loss / len(well_dataloaders[well_name]) 
            avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
            ax.set_xlabel('Day')
            ax.set_ylabel('Value')
            ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
            pdf.savefig(fig)
            plt.close()
    pdf.close()

In [66]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Get the best hyperparameters
best_params = study.best_params
print('Best hyperparameters:', best_params)

# Train the final model with the best hyperparameters
vector_length = best_params['vector_length']
num_layers = best_params['num_layers']
dropout = best_params['dropout']
lr = best_params['lr']
gamma = best_params['gamma']

[I 2024-07-10 15:10:24,804] A new study created in memory with name: no-name-9e7184b8-4ecb-4e1f-9052-7b364ed21a05
[W 2024-07-10 15:10:32,291] Trial 0 failed with parameters: {'vector_length': 1664, 'num_layers': 3, 'dropout': 0.4907871574431366, 'lr': 0.00021124431751369142, 'gamma': 0.9844411121769495} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/amrtamer/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/d8/xf5yjrpn5zvd1j56c42jys2w0000gn/T/ipykernel_806/3476955214.py", line 114, in objective
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
KeyboardInterrupt
[W 2024-07-10 15:10:32,295] Trial 0 failed with value None.


KeyboardInterrupt: 

In [120]:
vector_length = 1024
num_layers = 3
dropout = 0.19057012152026273
lr = 0.0001911914637460164
gamma = 0.9806898435934219 # 2nd best results on val set full training

In [72]:
vector_length = 336
num_layers = 3
dropout = 0.12831089625320793
lr = 0.0003724263259631224
gamma = 0.978770138138513 # best results on val set full training

In [73]:
# Example usage
context_size = days_window - 1
X_batch, Y_batch = next(iter(train_loader))
X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
print(f'The shape of input is: {X_batch.shape} and the shape of the output is {Y_batch.shape}')
model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
output, loss = model(X_batch, Y_batch)
print(f'The output shape of the LSTM will be {tuple(output.shape)} and a an example initial loss is {loss}')

The shape of input is: torch.Size([96, 5, 38]) and the shape of the output is torch.Size([96, 14])
The output shape of the LSTM will be (1344,) and a an example initial loss is 1.220238447189331


In [283]:
model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler

In [185]:
iterations = 30000

In [136]:
train_loader_iter = iter(train_loader)
log_file = get_next_log_file(path + "/Models/Global/LSTM/Logs")

log_message = f"Hyperparameters: Vector Length = {vector_length}, Number of Layers = {num_layers}, Dropout = {dropout}, Learning Rate = {lr}, Scheduler Gamma = {gamma}\n"

log_to_file(log_file, log_message)
model.train()
try:
    for i in range(iterations):
        model.train()  # Set model to training mode
        if i % eval_interval == 0 or i == iterations - 1:
            # Evaluate and save model parameters periodically
            save_params(model, optimizer, scheduler, 'current')  # Saving parameters
            model.eval()
            losses = estimate_loss(model)  # Estimate train and val loss
            model.train()
            log_message = f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
            print(log_message)
            log_to_file(log_file, log_message)  # Log to file
            if i != 0:
                scheduler.step()  # Perform scheduler step
        try:
            # Get the next batch
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            # Reinitialize the iterator if the DataLoader is exhausted
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)

        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model
        optimizer.zero_grad(set_to_none=True)  # Set gradients to 0 (to ensure gradients don't explode)
        loss.backward()  # Perform backpropagation
        optimizer.step()  # Perform parameter adjustment

except KeyboardInterrupt:
    print("Training ended manually by user.\n")
model.eval()
print("Training is complete")


step 0: train loss 1.1364, val loss 1.0948
step 500: train loss 0.0552, val loss 0.0535
step 1000: train loss 0.0515, val loss 0.0535
step 1500: train loss 0.0449, val loss 0.0485
step 2000: train loss 0.0427, val loss 0.0483
step 2500: train loss 0.0359, val loss 0.0467
step 3000: train loss 0.0293, val loss 0.0433
step 3500: train loss 0.0267, val loss 0.0392
step 4000: train loss 0.0218, val loss 0.0347
step 4500: train loss 0.0191, val loss 0.0335
step 5000: train loss 0.0168, val loss 0.0301
step 5500: train loss 0.0160, val loss 0.0296
step 6000: train loss 0.0145, val loss 0.0307
step 6500: train loss 0.0128, val loss 0.0271
step 7000: train loss 0.0120, val loss 0.0278
step 7500: train loss 0.0120, val loss 0.0261
step 8000: train loss 0.0100, val loss 0.0241
step 8500: train loss 0.0097, val loss 0.0250
step 9000: train loss 0.0088, val loss 0.0229
step 9500: train loss 0.0085, val loss 0.0239
step 10000: train loss 0.0076, val loss 0.0232
step 10500: train loss 0.0073, val lo

In [138]:
model.eval()
train_metrics = evaluate('train', model)
val_metrics = evaluate('val', model)
test_metrics = evaluate('test', model)
log_to_file(log_file, str(f"\nMetrics = {test_metrics}"))

Metrics on train set:

Loss is 0.0046
MAE is 0.0421
RMSE is 0.0680
R² is 0.9958


Metrics on val set:

Loss is 0.0217
MAE is 0.0762
RMSE is 0.1488
R² is 0.9788


Metrics on test set:

Loss is 0.0125
MAE is 0.0874
RMSE is 0.1123
R² is 0.4735




In [80]:
label_model_files(old_name='hi', new_name='best')

Files have been renamed successfully.


In [74]:
model, optimizer, scheduler = load_model('best')

In [81]:
predictions_with_history_graphs(model, visualization_test_loader, path + "/Models/Global/LSTM/Graphs")
predictions_graphs(model, visualization_test_loader, path + "/Models/Global/LSTM/Graphs")
all_predictions_graphs(model, data, data_test, path + "/Models/Global/LSTM/Graphs")
all_predictions_with_gaps_graphs(model, data, data_test, path + "/Models/Global/LSTM/Graphs")
test_metrics = evaluate('test', model)

Metrics on test set:

Loss is 0.0136
MAE is 0.0890
RMSE is 0.1168
R² is 0.4306




1. add time elapsed column
2. find a way to fill gaps
3. overfitting (add dropout)
5. deal with wht and whp
4. load model
evaulatiob=n on all e`xamples
6. Check if you need to further nromalize

should i adjust the data so that the lags are adjusted as if it hasnt seen it previously